In [46]:
import sys
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy import stats

In [142]:
def parse_loops_and_probs(fil1, fil2):
    """
    Parse two files and return a set of tuples containing the union of the first six elements
    from each file, and a dictionary mapping the first six elements to the 7th element of each file.
    """
    loop_set = set()
    probs1 = {}
    probs2 = {}
    
    with open(fil1, 'r') as source:
        for line in source:
            p = line.rstrip().split()
            loop_set.add(tuple(p[:6]))
            probs1[tuple(p[:6])] = p[7]
    
    with open(fil2, 'r') as source:
        for line in source:
            p = line.rstrip().split()
            loop_set.add(tuple(p[:6]))
            probs2[tuple(p[:6])] = p[7]
    
    return loop_set, probs1, probs2

def write_output(loop_set, probs1, probs2, output_file):
    """
    Write the output file with the union of loops and corresponding 7th column values from both files.
    """
    with open(output_file, 'w') as out:
        for loop in loop_set:
            # Get the 7th column value from each file, defaulting to 0 if not found
            val1 = probs1.get(loop, '0')
            val2 = probs2.get(loop, '0')
            # Write the loop and the 7th column values to the output file
            out.write('\t'.join(list(loop) + [val1, val2]) + '\n')

# Example usage in Jupyter Notebook
if __name__ == "__main__":
    # Replace 'file1.txt' and 'file2.txt' with the paths to your actual files
    infil1 = '/cluster/home/tmp/GBM/HiC/10loop/peakachu/diff/NPC-peakachu-5kb-loops.0.95.bedpe'
    infil2 = '/cluster/home/tmp/GBM/HiC/10loop/peakachu/diff/GBMmerge-peakachu-5kb-loops.0.95.bedpe'
    output_file = '/cluster/home/tmp/GBM/HiC/10loop/peakachu/diff/NPC_GBMmerge_merged.bedpe'
    
    loop_set, probs1, probs2 = parse_loops_and_probs(infil1, infil2)
    write_output(loop_set, probs1, probs2, output_file)

In [143]:
import sys
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy import stats

def quantile_norm(X):
    """
    Normalize the columns of X to each have the same distribution.
    """
    quantiles = np.mean(np.sort(X, axis=0), axis=1)
    ranks = np.apply_along_axis(stats.rankdata, 0, X)
    rank_indices = ranks.astype(int) - 1
    Xn = quantiles[rank_indices]
    return Xn

def parse_peakachu_loops(fil):
    L = []
    with open(fil, 'r') as source:
        for line in source:
            p = line.rstrip().split()
            key = (p[0], int(p[1]), int(p[2]), p[3], int(p[4]), int(p[5]))
            L.append(key)
    return L

def parse_probs(fil):
    L = []
    unique_1 = set()
    unique_2 = set()
    x = []
    y = []
    all_loops = []###修改
    x1 = []###修改
    y1 = []###修改
    with open(fil, 'r') as source:
        for line in source:
            p = line.rstrip().split()
            key = (p[0], int(p[1]), int(p[2]), p[3], int(p[4]), int(p[5]))
            all_loops.append(key) ###修改
            x1.append(float(p[6])) ###修改
            y1.append(float(p[7])) ###修改
            if (float(p[6]) > 0) and (float(p[7]) == 0):
                unique_1.add(key)
            elif (float(p[7]) > 0) and (float(p[6]) == 0):
                unique_2.add(key)
            else:
                L.append(key)
                x.append(float(p[6]))
                y.append(float(p[7]))
    x = np.r_[x]
    y = np.r_[y]
    return L, unique_1, unique_2, x, y, all_loops, x1, y1

def work_core(folds, fold1, fold2, union, loop_pool_1, loop_pool_2, thre=0.9):
    Pass = False
    for N in range(2, 11):
        model = GaussianMixture(N, covariance_type='full').fit(folds[:, np.newaxis])
        means = model.means_.ravel()
        ri = np.argmax(means)
        probs = model.predict_proba(folds[:, np.newaxis])[:, ri]
        idx = np.where(probs >= thre)[0]
        mask = folds[idx] < 1
        if mask.sum() == 0:
            Pass = True
            break
        else:
            if mask.sum() / mask.size < 0.01:
                Pass = True
                break

    unique1 = set()
    unique2 = set()
    if Pass:
        print('Number of Dists: {0}'.format(N))
        probs_1 = model.predict_proba(fold1[:, np.newaxis])[:, ri]
        probs_2 = model.predict_proba(fold2[:, np.newaxis])[:, ri]
        idx = np.where(probs_1 >= thre)[0]
        for i in idx:
            key = union[i]
            if (key in loop_pool_1) and (not key in loop_pool_2):
                unique1.add(key)
        idx = np.where(probs_2 >= thre)[0]
        for i in idx:
            key = union[i]
            if (key in loop_pool_2) and (not key in loop_pool_1):
                unique2.add(key)
    return unique1, unique2, model

def get_union_with_values(loop_pool_1, loop_pool_2, all_loops, x1, y1):
    """
    Produce a union of loops and retrieve corresponding fold1 and fold2 values.
    """
    loop_to_fold1 = {}
    loop_to_fold2 = {}

    # Create fold1 and fold2 from x1 and y1
    fold11 = np.array(x1) / np.array(y1)
    fold21 = np.array(y1) / np.array(x1)

    for i, loop in enumerate(all_loops):
        if loop in loop_pool_1 or loop in loop_pool_2:
            loop_to_fold1[loop] = fold11[i]
            loop_to_fold2[loop] = fold21[i]
    
    return loop_to_fold1, loop_to_fold2


loop_pool_1 = set(parse_peakachu_loops('/cluster/home/tmp/GBM/HiC/10loop/peakachu/diff/NPC-peakachu-5kb-loops.0.95.bedpe'))
loop_pool_2 = set(parse_peakachu_loops('/cluster/home/tmp/GBM/HiC/10loop/peakachu/diff/GBMmerge-peakachu-5kb-loops.0.95.bedpe'))
union, unique_1, unique_2, x, y, all_loops, x1, y1 = parse_probs('/cluster/home/tmp/GBM/HiC/10loop/peakachu/diff/NPC_GBMmerge_merged.bedpe')
thre = 0.95
repeat = 50

Xn = quantile_norm(np.r_['1,2,0', x, y])
x, y = Xn.T
fold1 = x / y
fold2 = y / x
folds = np.r_[fold1, fold2]

folds_c = folds.copy()
folds_c.sort()
cumsum = np.cumsum(np.diff(folds_c) < 2)
per_i = np.where(cumsum == np.arange(1, cumsum.size + 1))[-0][-1] + 1
per = folds_c[per_i]
folds = folds[folds <= per]

unique1, unique2, model = work_core(folds, fold1, fold2, union, loop_pool_1, loop_pool_2, thre=thre)
for i in range(repeat):
    u1, u2 = work_core(folds, fold1, fold2, union, loop_pool_1, loop_pool_2, thre=thre)[:2]
    unique1 = unique1 & u1
    unique2 = unique2 & u2
    if (not len(unique1)) and (not len(unique2)):
       break

unique1 = unique1 | unique_1
unique2 = unique2 | unique_2
unique1 = sorted(unique1)
unique2 = sorted(unique2)

# Get union with fold1 and fold2 values
loop_to_fold1, loop_to_fold2 = get_union_with_values(loop_pool_1, loop_pool_2, all_loops, x1, y1)

# # Output with fold values
outfil1 = '/cluster/home/tmp/GBM/HiC/10loop/peakachu/diff/GBMmerge.unique.loops'
with open(outfil1, 'w') as out:
    for line in unique1:
        tmp = list(map(str, line))
        fold_val = loop_to_fold1.get(line, 'NA')
        out.write('\t'.join(tmp) + '\t' + str(fold_val) + '\n')

outfil2 = '/cluster/home/tmp/GBM/HiC/10loop/peakachu/diff/NPC.unique.loops'
with open(outfil2, 'w') as out:
    for line in unique2:
        tmp = list(map(str, line))
        fold_val = loop_to_fold2.get(line, 'NA')
        out.write('\t'.join(tmp) + '\t' + str(fold_val) + '\n')


Number of Dists: 2
Number of Dists: 2


/tmp/ipykernel_105239/705246330.py:99: RuntimeWarning: divide by zero encountered in divide
  fold11 = np.array(x1) / np.array(y1)
/tmp/ipykernel_105239/705246330.py:100: RuntimeWarning: divide by zero encountered in divide
  fold21 = np.array(y1) / np.array(x1)
